In [ ]:
import numpy as np
import mne
import pickle
import matplotlib.pyplot as plt
from nice.algorithms.connectivity import epochs_compute_wsmi
np.set_printoptions(threshold=100)  # Default threshold

#load data
file_path = 'C:/Users/joshu/PartIIIProject/RSNNdale_attention_1_attention_test'
data = pickle.load(open(file_path, 'rb'))

attention_labels = data['label_attend'][0]
print("attend", attention_labels[:10])
label_left = data['label_left'][0]
print("label_left", label_left[:10])
label_right = data['label_right'][0]
print("label_right", label_right[:10])
attend_01 = data['attend'][0]
print ("attend01", attend_01[:10])


#not filtered for omitted trials 
left_input_LFP = data['LFP'][0][0]  # Left input  [0,1] means left, Right 
right_input_LFP = data['LFP'][0][1]  # Right input
attention_LFP = data['LFP_rec'][0][2]  # Attention layer  [2] means attention 

omitted = data["omit"][0]
# print("omitted", omitted[:10])

#trials where label_left and label_right differ, and trial isn't omitted
class_different_indices = np.where((label_left != label_right) & (omitted == 0))[0]


#subset further based on attention
left_indices = np.where((label_left != label_right) & (omitted == 0) & (attend_01 == 0))[0]
right_indices = np.where((label_left != label_right) & (omitted == 0) & (attend_01 == 1))[0]

print("class_different_indices", class_different_indices)
print("shape of class_different_indices", class_different_indices.shape)
print("shape of left_indices", left_indices.shape)
print("shape of right_indices", right_indices.shape)

left_input_LFP_om_left = left_input_LFP[left_indices]
right_input_LFP_om_left = right_input_LFP[left_indices]

left_input_LFP_om_right = left_input_LFP[right_indices]
right_input_LFP_om_right = right_input_LFP[right_indices]

attention_LFP_om_left = attention_LFP[left_indices]
attention_LFP_om_right = attention_LFP[right_indices]

# attend [11. 14. 18. 15.  2.  5. 12. 16.  2.  0.]
# label_left [11. 14.  6. 18.  2.  5.  6. 10.  3. 14.]
# label_right [15. 12. 18. 15. 12. 18. 12. 16.  2.  0.]
# omitted [0. 0. 1. 1. 0. 0. 1. 1. 1. 1.]


n_times = left_input_LFP.shape[1] ##=500
n_trials = left_input_LFP_om_left.shape[0]
print("n_samples", n_times)
print("n_trials", n_trials)

dt = 0.002
sfreq = 1 / dt


ch_names = ['left_input', 'right_input', 'attention_layer']
ch_types = ['eeg', 'eeg', 'eeg']
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)


#reshaping data for attention left
raw_data_left = np.concatenate([
    left_input_LFP_om_left, 
    right_input_LFP_om_left, 
    attention_LFP_om_left
], axis=0)  # Concatenate along time axis

# Reshape into (n_channels, n_samples)
raw_data_left = raw_data_left.reshape(3, -1)  # Now (3, 468*500) = (3, 234000)
print('raw data left shape:', raw_data_left.shape)  # (3, 234000)
print('raw data left, attending left, trial 0:', raw_data_left[0, :n_times])
#raw data left, attending left, trial 0: [ 0.          8.30495834  6.72716999 ... 75.45306396 79.49226379, 71.67674255]


raw_left = mne.io.RawArray(raw_data_left, info)



#reshaping date for attention right 
raw_data_right = np.concatenate([
    left_input_LFP_om_right,
    right_input_LFP_om_right,
    attention_LFP_om_right
], axis=0)

raw_data_right = raw_data_right.reshape(3, -1)
print('raw data right shape:', raw_data_right.shape)
print('raw data right, attending right, first right attending trial:', raw_data_right[1, 2*n_times:3*n_times])
#raw data right, attending right, first right attending trial: [  0.          16.3247776   16.00920868 ... 126.37789154 154.798873,  140.4855957 ]
#this is good, it is the same as in the single trial case, and seems to be stacking correctly


raw_right = mne.io.RawArray(raw_data_right, info)

#Creating RawArray with float64 data, n_channels=3, n_times=268000
#    Range : 0 ... 267999 =      0.000 ...   535.998 secs     why this time? seems to be out by 0.002 seconds of what would be expected

#defininf event objects, arrays like [0,0,1], [500, 0, 1], [1000, 0, 1] etc
events_left = np.array([[i * n_times, 0, 1] for i in range(len(left_input_LFP_om_left))])
events_right = np.array([[i * n_times, 0, 1] for i in range(len(right_input_LFP_om_right))])


print("events_left", events_left[:4])

epochs_left = mne.Epochs(raw_left, events_left, event_id={'Trial': 1}, tmin=0, tmax=(n_times - 1) / sfreq, baseline=None, preload=True)
epochs_right = mne.Epochs(raw_right, events_right, event_id={'Trial': 1}, tmin=0, tmax=(n_times - 1) / sfreq, baseline=None, preload=True)

kernel = 3
taus = [8, 16, 32, 64]  # in ms
wsmi_results = {'left': {}, 'right': {}}

for tau in taus:
    tau_samples = int(tau / (1000 / sfreq))
    print(f"tau_samples for {tau}: {tau_samples}")
    
    wsmi_left, _, _, _ = epochs_compute_wsmi(
        epochs_left, kernel=kernel, tau=tau_samples, backend='python', method_params={'bypass_csd': True}
    )
    wsmi_results['left'][tau] = wsmi_left
    #this containts the data for wsmi at a given tau given attending left. 

    wsmi_right, _, _, _ = epochs_compute_wsmi(
        epochs_right, kernel=kernel, tau=tau_samples, backend='python', method_params={'bypass_csd': True}
    )
    wsmi_results['right'][tau] = wsmi_right



print('wsmi_results.shape', wsmi_results['left'][8].shape)
#wsmi_results.shape (3, 3, 468), so get a 3x3 for each trial. note that values for performing a single trial are different to 
#extracting the wsmi value of the first trial from wsmi_results. unsure why. 

for tau_to_check in taus:

    # Extract the wSMI matrix for the first trial (attention left)
    wsmi_first_trial_left = wsmi_results['left'][tau_to_check][:, :, 0]  # 0 corresponds to the first trial

    # Print the result
    print(f"wSMI matrix for the first trial (attention left) at tau={tau_to_check} ms:\n", wsmi_first_trial_left)



# wSMI matrix for the first trial (attention left) at tau=8 ms:
#  [[0.         0.00228504 0.02923702]
#  [0.         0.         0.00223716]
#  [0.         0.         0.        ]]
# wSMI matrix for the first trial (attention left) at tau=16 ms:
#  [[0.         0.01912681 0.03944688]
#  [0.         0.         0.00618852]
#  [0.         0.         0.        ]]
# wSMI matrix for the first trial (attention left) at tau=32 ms:
#  [[0.         0.12345667 0.02594063]
#  [0.         0.         0.00655333]
#  [0.         0.         0.        ]]
# wSMI matrix for the first trial (attention left) at tau=64 ms:
#  [[0.         0.14327575 0.07129394]
#  [0.         0.         0.08184199]
#  [0.         0.         0.        ]]

c:\Users\joshu\nice\.venv\Lib\site-packages\mne\externals\tempita\__init__.py:35: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi


attend [11. 14. 18. 15.  2.  5. 12. 16.  2.  0.]
label_left [11. 14.  6. 18.  2.  5.  6. 10.  3. 14.]
label_right [15. 12. 18. 15. 12. 18. 12. 16.  2.  0.]
attend01 [0. 0. 1. 1. 0. 0. 1. 1. 1. 1.]
class_different_indices [   0    1    4 ... 2024 2029 2031]
shape of class_different_indices (1004,)
shape of left_indices (468,)
shape of right_indices (536,)
n_samples 500
n_trials 468
raw data left shape: (3, 234000)
raw data left, attending left, trial 0: [ 0.          8.30495834  6.72716999 ... 75.45306396 79.49226379
 71.67674255]
Creating RawArray with float64 data, n_channels=3, n_times=234000
    Range : 0 ... 233999 =      0.000 ...   467.998 secs
Ready.
raw data right shape: (3, 268000)
raw data right, attending right, first right attending trial: [  0.          16.3247776   16.00920868 ... 126.37789154 154.7988739
 140.4855957 ]
Creating RawArray with float64 data, n_channels=3, n_times=268000
    Range : 0 ... 267999 =      0.000 ...   535.998 secs
Ready.
events_left [[   0    0 

c:\Users\joshu\nice\.venv\Lib\site-packages\mne\epochs.py:406: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  selected = np.where(np.in1d(self.events[:, 2], values))[0]
c:\Users\joshu\nice\.venv\Lib\site-packages\mne\epochs.py:431: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  sub = np.where(np.in1d(selection, self.selection))[0]


No baseline correction applied
0 projection items activated
Loading data for 468 events and 500 original time points ...
0 bad epochs dropped
Not setting metadata
Not setting metadata
536 matching events found
No baseline correction applied
0 projection items activated
Loading data for 536 events and 500 original time points ...
0 bad epochs dropped
tau_samples for 8: 4
Cannot autodetect number of jobs


c:\Users\joshu\nice\.venv\Lib\site-packages\mne\epochs.py:406: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  selected = np.where(np.in1d(self.events[:, 2], values))[0]
c:\Users\joshu\nice\.venv\Lib\site-packages\mne\epochs.py:431: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  sub = np.where(np.in1d(selection, self.selection))[0]


Bypassing CSD
Filtering  at 41.67 Hz
Performing symbolic transformation
Running wsmi with python...
Cannot autodetect number of jobs
Bypassing CSD
Filtering  at 41.67 Hz
Performing symbolic transformation
Running wsmi with python...
tau_samples for 16: 8
Cannot autodetect number of jobs
Bypassing CSD
Filtering  at 20.83 Hz
Performing symbolic transformation
Running wsmi with python...
Cannot autodetect number of jobs
Bypassing CSD
Filtering  at 20.83 Hz
Performing symbolic transformation
Running wsmi with python...
tau_samples for 32: 16
Cannot autodetect number of jobs
Bypassing CSD
Filtering  at 10.42 Hz
Performing symbolic transformation
Running wsmi with python...
Cannot autodetect number of jobs
Bypassing CSD
Filtering  at 10.42 Hz
Performing symbolic transformation
Running wsmi with python...
tau_samples for 64: 32
Cannot autodetect number of jobs
Bypassing CSD
Filtering  at 5.21 Hz
Performing symbolic transformation
Running wsmi with python...


In [ ]:
import numpy as np
import mne
import pickle
import matplotlib.pyplot as plt
from nice.algorithms.connectivity import epochs_compute_wsmi
np.set_printoptions(threshold=100)  #default threshold

#load data
file_path = 'C:/Users/joshu/PartIIIProject/RSNNdale_attention_1_attention_test'
data = pickle.load(open(file_path, 'rb'))

attention_labels = data['label_attend'][0]
label_left = data['label_left'][0]
label_right = data['label_right'][0]
attend_01 = data['attend'][0]


#not filtered for omitted trials 
left_input_LFP = data['LFP'][0][0]  # Left input  [0,1] means left, Right 
right_input_LFP = data['LFP'][0][1]  # Right input
attention_LFP = data['LFP_rec'][0][2]  # Attention layer  [2] means attention 
omitted = data["omit"][0]


# attend [11. 14. 18. 15.  2.  5. 12. 16.  2.  0.]
# label_left [11. 14.  6. 18.  2.  5.  6. 10.  3. 14.]
# label_right [15. 12. 18. 15. 12. 18. 12. 16.  2.  0.]
#attend01 [0. 0. 1. 1. 0. 0. 1. 1. 1. 1.]
#omitted [0. 0. 1. 1. 0. 1. 0. 1. 1. 1.]


trial_idx = 0
left_input_trial = left_input_LFP[trial_idx]
right_input_trial= right_input_LFP[trial_idx]
attention_trial = attention_LFP[trial_idx]

print("left_input_trial", left_input_trial[0:20])
print("right_input_trial", right_input_trial[0:20])
print("attention_trial", attention_trial[0:20])


raw_data_raw = np.stack([left_input_trial, right_input_trial, attention_trial])
print(raw_data_raw.shape)
print(raw_data_raw)
print('right, attending right, trial 14:', raw_data_raw[1])
#right, attending right, trial 14: [   0.           16.3247776    16.00920868   20.1116848...

n_times = raw_data_raw.shape[1] ##=500
print("n_samples", n_times)

dt = 0.002
sfreq = 1 / dt
ch_names = ['left_input', 'right_input', 'attention_layer']
ch_types = ['eeg', 'eeg', 'eeg']
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)


raw_data = mne.io.RawArray(raw_data_raw, info)

events = np.array([
    [0,0,1]
])


epochs_raw = mne.Epochs(raw_data, events, event_id={'Trial': 1}, tmin=0, tmax=(n_times - 1) / sfreq, baseline=None, preload=True)

print("Raw data shape:", raw_data.get_data().shape)
print("epochs_raw shape:", epochs_raw.get_data().shape)
print("Events array:", events)
print("Number of epochs:", len(epochs_raw))


kernel = 3
taus = [8, 16, 32, 64]  # in ms
#making wsmi results dictionary to store raw, normalised, and smoothed data
wsmi_results = {}

for tau in taus:
    tau_samples = int(tau / (1000 / sfreq))
    print(f"tau_samples for {tau}: {tau_samples}")
    
    wsmi, _, _, _ = epochs_compute_wsmi(
        epochs_raw, kernel=kernel, tau=tau_samples, backend='python', method_params={'bypass_csd': True}
    )

    wsmi_results[tau] = wsmi


for tau in taus:

    print("wsmi results raw", wsmi_results[tau][:, :, 0])

# wsmi results raw [[0.         0.00236065 0.02781281]
#  [0.         0.         0.00095507]
#  [0.         0.         0.        ]]
# wsmi results raw [[0.         0.01833334 0.02831222]
#  [0.         0.         0.00670613]
#  [0.         0.         0.        ]]
# wsmi results raw [[0.         0.11334197 0.00812583]
#  [0.         0.         0.01633531]
#  [0.         0.         0.        ]]
# wsmi results raw [[0.         0.12664354 0.10996675]
#  [0.         0.         0.10764085]
#  [0.         0.         0.        ]]

In [ ]:
import numpy as np
import mne
import pickle
import matplotlib.pyplot as plt
from nice.algorithms.connectivity import epochs_compute_wsmi
np.set_printoptions(threshold=900)  #default threshold

#load data
file_path = 'C:/Users/joshu/PartIIIProject/RSNNdale_attention_1_attention_test'
data = pickle.load(open(file_path, 'rb'))

attention_labels = data['label_attend'][0]
print("attend", attention_labels[:10])
label_left = data['label_left'][0]
print("label_left", label_left[:10])
label_right = data['label_right'][0]
print("label_right", label_right[:10])
attend_01 = data['attend'][0]
print ("attend01", attend_01[:10])


#not filtered for omitted trials 
left_input_LFP = data['LFP'][0][0]  # Left input  [0,1] means left, Right 
right_input_LFP = data['LFP'][0][1]  # Right input
attention_LFP = data['LFP_rec'][0][2]  # Attention layer  [2] means attention 

omitted = data["omit"][0]
print("omitted", omitted[:10])

# attend [11. 14. 18. 15.  2.  5. 12. 16.  2.  0.]
# label_left [11. 14.  6. 18.  2.  5.  6. 10.  3. 14.]
# label_right [15. 12. 18. 15. 12. 18. 12. 16.  2.  0.]
#attend01 [0. 0. 1. 1. 0. 0. 1. 1. 1. 1.]
#omitted [0. 0. 1. 1. 0. 1. 0. 1. 1. 1.]

# attend [11. 14.  2. 12.]
# label_left [11. 14.  2. 6.]
# label_right [15. 12. 12. 12.]
#attend01 [0. 0.  0. 1.]

#plotting just the first trials data
trial_idx = 0
plt.plot(left_input_LFP[trial_idx], label="Left Input LFP")
plt.plot(right_input_LFP[trial_idx], label="Right Input LFP")
plt.plot(attention_LFP[trial_idx], label="Attention Layer LFP")
plt.title(f"LFP Data for Trial {trial_idx}")
plt.xlabel("Time (ms)")
plt.ylabel("LFP")
plt.legend()
plt.show()


#of non omitted trials
#first left 
#second left
#third left
#fourth right 

#therefore expect greater wsmi on left for 1,2,3, and greater wsmi on right for 4

left_input_triali = left_input_LFP[trial_idx]
right_input_triali= right_input_LFP[trial_idx]
attention_triali = attention_LFP[trial_idx]

def min_max_normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# Normalize each channel
left_input_normalized = min_max_normalize(left_input_triali)
right_input_normalized = min_max_normalize(right_input_triali)
attention_normalized = min_max_normalize(attention_triali)

plt.plot(left_input_normalized, label="Left Input LFP")
plt.plot(right_input_normalized, label="Right Input LFP")
plt.plot(attention_normalized, label="Attention Layer LFP")
plt.title(f"Normalised LFP Data for Trial {trial_idx}")
plt.xlabel("Time (ms)")
plt.ylabel("LFP")
plt.legend()
plt.show()

# Apply smoothing (e.g., Gaussian filter)
from scipy.ndimage import gaussian_filter1d

sigma = 5  # Standard deviation for Gaussian kernel
left_input_smoothed = gaussian_filter1d(left_input_normalized, sigma)
right_input_smoothed = gaussian_filter1d(right_input_normalized, sigma)
attention_smoothed = gaussian_filter1d(attention_normalized, sigma)

plt.plot(left_input_smoothed, label="Left Input LFP")
plt.plot(right_input_smoothed, label="Right Input LFP")
plt.plot(attention_smoothed, label="Attention Layer LFP")
plt.title(f"Smoothed LFP Data for Trial {trial_idx}")
plt.xlabel("Time (ms)")
plt.ylabel("LFP")
plt.legend()
plt.show()

raw_data_raw = np.stack([left_input_triali, right_input_triali, attention_triali])
raw_data_normalised = np.stack([left_input_normalized, right_input_normalized, attention_normalized])
raw_data_smoothed = np.stack([left_input_smoothed, right_input_smoothed, attention_smoothed])


n_times = raw_data_raw.shape[1] ##=500
print("n_samples", n_times)

dt = 0.002
sfreq = 1 / dt
ch_names = ['left_input', 'right_input', 'attention_layer']
ch_types = ['eeg', 'eeg', 'eeg']
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)


raw_data = mne.io.RawArray(raw_data_raw, info)
raw_data_normalised = mne.io.RawArray(raw_data_normalised, info)
raw_data_smoothed = mne.io.RawArray(raw_data_smoothed, info)


events = np.array([
    [0,0,1]
])


epochs_raw = mne.Epochs(raw_data, events, event_id={'Trial': 1}, tmin=0, tmax=(n_times - 1) / sfreq, baseline=None, preload=True)
epochs_normalised = mne.Epochs(raw_data_normalised, events, event_id={'Trial': 1}, tmin=0, tmax=(n_times - 1) / sfreq, baseline=None, preload=True)
epochs_smoothed = mne.Epochs(raw_data_smoothed, events, event_id={'Trial': 1}, tmin=0, tmax=(n_times - 1) / sfreq, baseline=None, preload=True)

print("Raw data shape:", raw_data.get_data().shape)
print("Events array:", events)
print("Number of epochs:", len(epochs_raw))


kernel = 3
taus = [8, 16, 32, 64]  # in ms
#making wsmi results dictionary to store raw, normalised, and smoothed data
wsmi_results = {'raw': {}, 'normalised': {}, 'smoothed': {}}

for tau in taus:
    tau_samples = int(tau / (1000 / sfreq))
    print(f"tau_samples for {tau}: {tau_samples}")
    
    wsmi, _, _, _ = epochs_compute_wsmi(
        epochs_raw, kernel=kernel, tau=tau_samples, backend='python', method_params={'bypass_csd': True}
    )

    wsmi_results['raw'][tau] = wsmi

    wsmi, _, _, _ = epochs_compute_wsmi(
        epochs_normalised, kernel=kernel, tau=tau_samples, backend='python', method_params={'bypass_csd': True}
    )

    wsmi_results['normalised'][tau] = wsmi

    wsmi, _, _, _ = epochs_compute_wsmi(
        epochs_smoothed, kernel=kernel, tau=tau_samples, backend='python', method_params={'bypass_csd': True}
    )

    wsmi_results['smoothed'][tau] = wsmi

print("wsmi results raw", wsmi_results['raw'][8])


# Initialize lists to store wSMI values
wsmi_left_attention_raw = []  # wSMI between left_input and attention_layer
wsmi_right_attention_raw = []  # wSMI between right_input and attention_layer

wsmi_left_attention_normalised = []  # wSMI between left_input and attention_layer
wsmi_right_attention_normalised = []  # wSMI between right_input and attention_layer

wsmi_left_attention_smoothed = []  # wSMI between left_input and attention_layer
wsmi_right_attention_smoothed = []  # wSMI between right_input and attention_layer

# Extract wSMI values for each tau
for tau in taus:
    wsmi_left_attention_raw.append(wsmi_results['raw'][tau][0, 2, 0])  # left_input vs attention_layer
    wsmi_right_attention_raw.append(wsmi_results['raw'][tau][1, 2, 0])  # right_input vs attention_layer

    wsmi_left_attention_normalised.append(wsmi_results['normalised'][tau][0, 2, 0])  # left_input vs attention_layer
    wsmi_right_attention_normalised.append(wsmi_results['normalised'][tau][1, 2, 0])  # right_input vs attention_layer

    wsmi_left_attention_smoothed.append(wsmi_results['smoothed'][tau][0, 2, 0])  # left_input vs attention_layer
    wsmi_right_attention_smoothed.append(wsmi_results['smoothed'][tau][1, 2, 0])  # right_input vs attention_layer

# Plot the results
fig, axs = plt.subplots(3, 1, figsize=(6, 15))

# Raw data plot
axs[0].scatter(taus, wsmi_left_attention_raw, marker='o', label='Left Input vs Attention Layer', color='blue')
axs[0].scatter(taus, wsmi_right_attention_raw, marker='o', label='Right Input vs Attention Layer', color='orange')
axs[0].set_xlabel('Tau (ms)', fontsize=12)
axs[0].set_ylabel('wSMI', fontsize=12)
axs[0].set_title('Raw Data: wSMI vs Tau for Input vs Attention Layer', fontsize=14)
axs[0].legend(fontsize=10)
axs[0].grid(True)

# Normalised data plot
axs[1].scatter(taus, wsmi_left_attention_normalised, marker='o', label='Left Input vs Attention Layer', color='blue')
axs[1].scatter(taus, wsmi_right_attention_normalised, marker='o', label='Right Input vs Attention Layer', color='orange')
axs[1].set_xlabel('Tau (ms)', fontsize=12)
axs[1].set_ylabel('wSMI', fontsize=12)
axs[1].set_title('Normalised Data: wSMI vs Tau for Input vs Attention Layer', fontsize=14)
axs[1].legend(fontsize=10)
axs[1].grid(True)

# Smoothed data plot
axs[2].scatter(taus, wsmi_left_attention_smoothed, marker='o', label='Left Input vs Attention Layer', color='blue')
axs[2].scatter(taus, wsmi_right_attention_smoothed, marker='o', label='Right Input vs Attention Layer', color='orange')
axs[2].set_xlabel('Tau (ms)', fontsize=12)
axs[2].set_ylabel('wSMI', fontsize=12)
axs[2].set_title('Smoothed Data: wSMI vs Tau for Input vs Attention Layer', fontsize=14)
axs[2].legend(fontsize=10)
axs[2].grid(True)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


##person correlations
from scipy.stats import pearsonr

# Compute correlation for left_input and attention_layer
corr_left_attention, _ = pearsonr(left_input_triali, attention_triali)
print(f"Pearson correlation (Left Input vs. Attention Layer): {corr_left_attention}")

# Compute correlation for right_input and attention_layer
corr_right_attention, _ = pearsonr(right_input_triali, attention_triali)
print(f"Pearson correlation (Right Input vs. Attention Layer): {corr_right_attention}")




In [ ]:

from scipy.stats import pearsonr


attleft_pearson_left = []
attleft_pearson_right = []
attright_pearson_left = []
attright_pearson_right = []

for i in range(len(left_indices)):
    corr_left, _ = pearsonr(left_input_LFP_om_left[i], attention_LFP_om_left[i])
    attleft_pearson_left.append(corr_left)
    corr_right, _ = pearsonr(right_input_LFP_om_left[i], attention_LFP_om_left[i])
    attleft_pearson_right.append(corr_right)

for i in range(len(right_indices)):
    ccrr_left, _ = pearsonr(left_input_LFP_om_right[i], attention_LFP_om_right[i])
    attright_pearson_left.append(corr_left)
    corr_right, _ = pearsonr(right_input_LFP_om_right[i], attention_LFP_om_right[i])
    attright_pearson_right.append(corr_right)


mean_corr_left_attleft = np.mean(attleft_pearson_left)
mean_corr_right_attleft = np.mean(attleft_pearson_right)
mean_corr_left_attright = np.mean(attright_pearson_left)
mean_corr_right_attright = np.mean(attright_pearson_right)

print("mean_corr_left_attleft", mean_corr_left_attleft)
print("mean_corr_right_attleft", mean_corr_right_attleft)
print("mean_corr_left_attright", mean_corr_left_attright)
print("mean_corr_right_attright", mean_corr_right_attright)



#with smoothing

from scipy.ndimage import gaussian_filter1d

sigma = 3  # Standard deviation for Gaussian kernel
left_input_smoothed_att_left = gaussian_filter1d(left_input_LFP_om_left, sigma)
left_input_smoothed_att_right = gaussian_filter1d(left_input_LFP_om_right, sigma)
right_input_smoothed_att_left = gaussian_filter1d(right_input_LFP_om_left, sigma)
right_input_smoothed_att_right = gaussian_filter1d(right_input_LFP_om_right, sigma)
attention_smoothed_att_left = gaussian_filter1d(attention_LFP_om_left, sigma)
attention_smoothed_att_right = gaussian_filter1d(attention_LFP_om_right, sigma)


attleft_pearson_left = []
attleft_pearson_right = []
attright_pearson_left = []
attright_pearson_right = []

for i in range(len(left_indices)):
    corr_left, _ = pearsonr(left_input_smoothed_att_left[i], attention_smoothed_att_left[i])
    attleft_pearson_left.append(corr_left)
    corr_right, _ = pearsonr(right_input_smoothed_att_left[i], attention_smoothed_att_left[i])
    attleft_pearson_right.append(corr_right)

for i in range(len(right_indices)):
    corr_left, _ = pearsonr(left_input_smoothed_att_right[i], attention_smoothed_att_right[i])
    attright_pearson_left.append(corr_left)
    corr_right, _ = pearsonr(right_input_smoothed_att_right[i], attention_smoothed_att_right[i])
    attright_pearson_right.append(corr_right)


mean_corr_left_attleft = np.mean(attleft_pearson_left)
mean_corr_right_attleft = np.mean(attleft_pearson_right)
mean_corr_left_attright = np.mean(attright_pearson_left)
mean_corr_right_attright = np.mean(attright_pearson_right)

print("mean_corr_left_attleft", mean_corr_left_attleft)
print("mean_corr_right_attleft", mean_corr_right_attleft)
print("mean_corr_left_attright", mean_corr_left_attright)
print("mean_corr_right_attright", mean_corr_right_attright)

